In [1]:
include("./structured_grid_uniform.jl")
include("./constant.jl")
include("./controls.jl")
include("./EOS.jl")
include("./transport.jl")
include("./momentum.jl")
include("./pressure.jl")


using Plots
#using PlotlyJS
using LinearAlgebra
using SparseArrays
using IterativeSolvers

using Pardiso

function main()

    Nx = 2
    Ny = 2
    Nz = 1
    Lx = 1.0
    Ly = 1.0
    Lz = 0.1
    realMaxIter = 1000000
    pseudoMaxIter = 30
    pseudoMaxResidual = -4.0

    CFL = 0.5
    Δt = 1.e-6
    Lco = 1.0
    Uco = 1.0

    👉 = controls(
        Nx,Ny,Nz, Lx,Ly,Lz, 
        realMaxIter,pseudoMaxIter,pseudoMaxResidual, 
        CFL, Δt, Lco, Uco,
        0.0, 0, 0, 0.0, 
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
        21,22,23,24,25,26,27,28
    )

    cells = Vector{mesh.Cell}(undef, 0)
    faces = Vector{mesh.Face}(undef, 0)
    faces_internal = Vector{mesh.Face}(undef, 0)
    faces_boundary = Vector{mesh.Face}(undef, 0)
    faces_boundary_top = Vector{mesh.Face}(undef, 0)
    faces_boundary_bottom = Vector{mesh.Face}(undef, 0)
    faces_boundary_left = Vector{mesh.Face}(undef, 0)
    faces_boundary_right = Vector{mesh.Face}(undef, 0)

    structured_grid_uniform!(
        👉,
        cells,
        faces,
        faces_internal,
        faces_boundary,
        faces_boundary_top,
        faces_boundary_bottom,
        faces_boundary_left,
        faces_boundary_right
    )

    # initialization
    for cell in cells
        cell.var[👉.p] = 101325.0
        cell.var[👉.u] = 0.0
        cell.var[👉.v] = 0.0
        cell.var[👉.w] = 0.0
        cell.var[👉.T] = 300.0
        cell.var[👉.Y₁] = 0.0

        if cell.x < 0.5 && cell.y < 0.5
            cell.var[👉.Y₁] = 1.0
        end

        
        cell.var[👉.pⁿ] = cell.var[👉.p]
        cell.var[👉.uⁿ] = cell.var[👉.u]
        cell.var[👉.vⁿ] = cell.var[👉.v]
        cell.var[👉.wⁿ] = cell.var[👉.w]
        cell.var[👉.Y₁ⁿ] = cell.var[👉.Y₁]
        cell.var[👉.Y₂ⁿ] = cell.var[👉.Y₂]
    end

    # EOS
    EOS!(👉, cells)
    
    # Transport
    transport!(👉, cells)


    # solver
    #=
    NSeq(
        👉,
        cells,
        faces,
        faces_internal,
        faces_boundary,
        faces_boundary_top,
        faces_boundary_bottom,
        faces_boundary_left,
        faces_boundary_right
    )
    =#

    momentum!(
        👉,
        cells,
        faces,
        faces_internal,
        faces_boundary,
        faces_boundary_top,
        faces_boundary_bottom,
        faces_boundary_left,
        faces_boundary_right
    )

    println("momentum equation success")
    
    pressure!(
        👉,
        cells,
        faces,
        faces_internal,
        faces_boundary,
        faces_boundary_top,
        faces_boundary_bottom,
        faces_boundary_left,
        faces_boundary_right
    )

    println("pressure equation success")









    
    # Plotting
    X = zeros(Float64, length(cells), 1)
    Y = zeros(Float64, length(cells), 1)
    VAR = zeros(Float64, length(cells), 6)
    for i in 1:length(cells)
        X[i] = cells[i].x
        Y[i] = cells[i].y
        VAR[i,1] = cells[i].var[👉.p]
        VAR[i,2] = cells[i].var[👉.u]
        VAR[i,3] = cells[i].var[👉.T]
        VAR[i,4] = cells[i].var[👉.Y₁]
        VAR[i,5] = cells[i].var[👉.ρ]
        VAR[i,6] = cells[i].var[👉.c]
        
    end
    #plt = plot(X,VAR,layout = 
    #grid(3, 2),
    #label = ["p" "u" "T" "Y₁" "ρ" "c"] )
    #plot(plt)
    #contourf!(X,Y,VAR)

    Δx = 👉.Lx/👉.Nx
    Δy = 👉.Ly/👉.Ny 
    Δz = 👉.Lz/👉.Nz 

    VAR2 = zeros(Float64, Nx, Ny)
    for i in 1:Nx
        for j in 1:Ny
            k=1
            ijk = i + 👉.Nx*(j-1) + 👉.Nx*👉.Ny*(k-1)
            VAR2[i,j] = cells[ijk].var[👉.ρ]
        end
    end
    plot(contour(
        x=0.5*Δx:Δx:👉.Lx,#X, # horizontal axis
        y=0.5*Δy:Δy:👉.Ly,#Y, # vertical axis
        z=VAR2'#VAR[:,5]'
    ))

    sleep(100.0)

end






# calculation main
main()




momentum equation success
pressure equation success

In [25]:

# calculation main
main()

MethodError: MethodError: no method matching structured_grid_uniform!(::controls, ::Vector{Main.mesh.Cell}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face}, ::Vector{Main.mesh.Face})
Closest candidates are:
  structured_grid_uniform!(::controls, !Matched::Vector{Main.mesh.Cell}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}) at e:\[[ 영린 관련 폴더 ]]\[[ code ]]\Incompressible_Pressure_Based_with_Julia\structured_grid_uniform.jl:5
  structured_grid_uniform!(::controls, !Matched::Vector{Main.mesh.Cell}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Face}, !Matched::Vector{Main.mesh.Fac

In [15]:
function NSeq(
    👉::controls,
    cell::Vector{mesh.Cell},
    face::Vector{mesh.Face},
    face_internal::Vector{mesh.Face},
    face_boundary::Vector{mesh.Face},
    face_boundary_top::Vector{mesh.Face},
    face_boundary_bottom::Vector{mesh.Face},
    face_boundary_left::Vector{mesh.Face},
    face_boundary_right::Vector{mesh.Face}
    )
    
    # solver
    👉.time = 0.0
    👉.realIter = 1
    total_iter = 1.0

    while(
        👉.realIter <= 👉.realMaxIter
    )
        println("real-time Step: $(👉.realIter) \t Time: $(👉.time)")

        # Qⁿ, Qⁿ⁻¹
        if 👉.realIter == 1
            update_real_conservative0!(👉, cells)
        else
            update_real_conservative!(👉, cells)
        end

        👉.pseudoIter = 1
        👉.residual = 10000.0
        residual0 = 10000.0
        while(
            👉.pseudoIter ≤ 👉.pseudoMaxIter &&
            👉.residual-residual0 ≥ 👉.pseudoMaxResidual
        )
            # momentum
            

            # pressure

            # volume fraction

            # sparse A matrix
            A = zeros(Float64, length(cells), 6, 6)
            #construct_A_matrix_implicit!(👉, A, cells, faces)
            construct_A_matrix_explicit!(👉, A, cells)

            # Ax=B
            x = zeros(Float64, length(cells), 6)
            #linear_solver_implicit!(A, x, B)
            linear_solver_explicit!(A, x, B)

            # residual norm
            👉.residual = residual_norm!(x, cells)
            if 👉.pseudoIter == 1
                residual0 = 👉.residual
            end
            
            # update primitive
            update_primitive!(👉, x, cells)
            
            # EOS
            EOS!(👉, cells)
            
            # residual print
            println("- pseudo-time Step: $(👉.pseudoIter) \t",
            "log₁₀|ΔR|₂: $(round((👉.residual-residual0),digits=8))")

            👉.pseudoIter += 1
            total_iter += 1.0

        end

        👉.realIter += 1
        👉.time += 👉.Δt

    end
end

NSeq (generic function with 1 method)

In [ ]:

function plotting()
    # Plotting
    X = zeros(Float64, length(cells), 1)
    Y = zeros(Float64, length(cells), 1)
    VAR = zeros(Float64, length(cells), 6)
    for i in 1:length(cells)
        X[i] = cells[i].x
        Y[i] = cells[i].y
        VAR[i,1] = cells[i].var[👉.p]
        VAR[i,2] = cells[i].var[👉.u]
        VAR[i,3] = cells[i].var[👉.T]
        VAR[i,4] = cells[i].var[👉.Y₁]
        VAR[i,5] = cells[i].var[👉.ρ]
        VAR[i,6] = cells[i].var[👉.c]
        
    end
    #plt = plot(X,VAR,layout = 
    #grid(3, 2),
    #label = ["p" "u" "T" "Y₁" "ρ" "c"] )
    #plot(plt)
    #contourf!(X,Y,VAR)

    Δx = 👉.Lx/👉.Nx
    Δy = 👉.Ly/👉.Ny 
    Δz = 👉.Lz/👉.Nz 

    VAR2 = zeros(Float64, Nx, Ny)
    for i in 1:Nx
        for j in 1:Ny
            k=1
            ijk = i + 👉.Nx*(j-1) + 👉.Nx*👉.Ny*(k-1)
            VAR2[i,j] = cells[ijk].var[👉.ρ]
        end
    end
    plot(contour(
        x=0.5*Δx:Δx:👉.Lx,#X, # horizontal axis
        y=0.5*Δy:Δy:👉.Ly,#Y, # vertical axis
        z=VAR2'#VAR[:,5]'
    ))
    #data = contour(;z=VAR, x=X, y=Y)
end

In [ ]:

# calculation main
main()

# plotting
plotting()